In [1]:
import os, sys, glob
import numpy as np
np.seterr(divide='ignore') # ignode divide by zero warnings
import astropy.io.fits as fits
from astropy.table import Table
import fitsio
import healpy as hp
from astropy.coordinates import SkyCoord
import astropy.units as units
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import warnings; warnings.simplefilter('ignore')

sys.path.append('/global/homes/q/qmxp55/DESI/omarlibs')
from main_def import get_random, flux_to_mag, get_sweep_whole, bgsmask, getGeoCuts
from photometric_def import get_stars, get_galaxies
import raichoorlib

# add GRAPHVIZ bin files to PATH, otherwise it doesn't find them
os.environ['PATH'] = '/global/u2/q/qmxp55/bin'

In [2]:
#input files

inptfiles = {}

#inptfiles['dr8randroot']= '/project/projectdirs/desi/target/catalogs/dr8/0.31.0/randoms/randoms-inside-dr8-0.31.0-'
inptfiles['dr8pix']     = '/project/projectdirs/desi/target/catalogs/dr8/0.31.1/pixweight/pixweight-dr8-0.31.1.fits'
inptfiles['desitile']   = '/global/cscratch1/sd/raichoor/desi-tiles-viewer.fits' # from Eddie, see [desi-survey 647]
inptfiles['bgsdr8']   = '/global/cscratch1/sd/qmxp55/bgs_dr8_0.31.1+cuts.npy'
inptfiles['bgssvtiles'] = '/global/cscratch1/sd/qmxp55/BGS_SV_30_3x_superset60_JUL2019.fits'
inptfiles['bgsdr8relax'] = '/global/cscratch1/sd/qmxp55/bgs_dr8_0.31.1_relaxed.npy'
inptfiles['dr8maskbitsource'] = '/global/cscratch1/sd/qmxp55/sweep_files/dr8_sweep_whole_maskbitsource.npy'
#inptfiles['bgsdr8relax-north'] = '/global/cscratch1/sd/qmxp55/desitarget_output/targets-BGS-main-dr8.0-1-north.fits'
#inptfiles['bgsdr8relax-south'] = '/global/cscratch1/sd/qmxp55/desitarget_output/targets-BGS-main-dr8.0-1-south.fits'

for file in inptfiles.keys():
    print(file, '\t=', inptfiles[file])
    
    
#settings
org          = 120  # centre ra for mollweide plots
Nranfiles = 3
veto_maskbits= [1, 12, 13] #BGS only applies the BS masking
dec_resol_ns = 32.375 # ADM dec to split north/south
projection   = 'mollweide'
# for healpy
hdr          = fits.getheader(inptfiles['dr8pix'],1)
nside,nest   = hdr['hpxnside'],hdr['hpxnest']
npix         = hp.nside2npix(nside)
pixarea      = hp.nside2pixarea(nside,degrees=True)

dr8pix 	= /project/projectdirs/desi/target/catalogs/dr8/0.31.1/pixweight/pixweight-dr8-0.31.1.fits
desitile 	= /global/cscratch1/sd/raichoor/desi-tiles-viewer.fits
bgsdr8 	= /global/cscratch1/sd/qmxp55/bgs_dr8_0.31.1+cuts.npy
bgssvtiles 	= /global/cscratch1/sd/qmxp55/BGS_SV_30_3x_superset60_JUL2019.fits
bgsdr8relax 	= /global/cscratch1/sd/qmxp55/bgs_dr8_0.31.1_relaxed.npy
dr8maskbitsource 	= /global/cscratch1/sd/qmxp55/sweep_files/dr8_sweep_whole_maskbitsource.npy


In [3]:
# is in desi nominal footprint? (using tile radius of 1.6 degree)
# small test shows that it broadly works to cut on desi footprint 
def get_isdesi(ra,dec):
    radius   = 1.6 # degree
    tmpnside = 16
    tmpnpix  = hp.nside2npix(tmpnside)
    # first reading desi tiles, inside desi footprint (~14k deg2)
    hdu  = fits.open(inptfiles['desitile'])
    data = hdu[1].data
    keep = (data['in_desi']==1)
    data = data[keep]
    tra,tdec = data['ra'],data['dec']
    # get hppix inside desi tiles
    theta,phi  = hp.pix2ang(tmpnside,np.arange(tmpnpix),nest=nest)
    hpra,hpdec = 180./np.pi*phi,90.-180./np.pi*theta
    hpindesi   = np.zeros(tmpnpix,dtype=bool)
    _,ind,_,_,_= raichoorlib.search_around(tra,tdec,hpra,hpdec,search_radius=1.6*3600)
    hpindesi[np.unique(ind)] = True
    ## small hack to recover few rejected pixels inside desi. Avoid holes if any
    tmp  = np.array([i for i in range(tmpnpix) 
                     if hpindesi[hp.get_all_neighbours(tmpnside,i,nest=nest)].sum()==8])
    hpindesi[tmp] = True
    ##
    pixkeep    = np.where(hpindesi)[0]
    # now compute the hppix for the tested positions
    pix  = hp.ang2pix(tmpnside,(90.-dec)*np.pi/180.,ra*np.pi/180.,nest=nest)
    keep = np.in1d(pix,pixkeep)
    return keep

def get_isbgstile(ra,dec):
    radius   = 1.6 # degree
    tmpnside = 256
    tmpnpix  = hp.nside2npix(tmpnside)
    # first reading desi tiles, inside desi footprint (~14k deg2)
    hdu  = fits.open(inptfiles['bgssvtiles'])
    data = hdu[1].data
    #keep = (data['in_desi']==1)
    #data = data[keep]
    tra,tdec = data['RA'],data['DEC']
    # get hppix inside desi tiles
    theta,phi  = hp.pix2ang(tmpnside,np.arange(tmpnpix),nest=nest)
    hpra,hpdec = 180./np.pi*phi,90.-180./np.pi*theta
    hpindesi   = np.zeros(tmpnpix,dtype=bool)
    
    idx,ind,_,_,_= raichoorlib.search_around(tra,tdec,hpra,hpdec,search_radius=1.6*3600)
    
    hpindesi[np.unique(ind)] = True
    ## small hack to recover few rejected pixels inside desi. Avoid holes if any
    #tmp  = np.array([i for i in range(tmpnpix) 
    #                 if hpindesi[hp.get_all_neighbours(tmpnside,i,nest=nest)].sum()==8])
    #hpindesi[tmp] = True
    ##
    pixkeep    = np.where(hpindesi)[0]
    # now compute the hppix for the tested positions
    pix  = hp.ang2pix(tmpnside,(90.-dec)*np.pi/180.,ra*np.pi/180.,nest=nest)
    keep = np.in1d(pix,pixkeep)
    
    hptileid = np.ones(tmpnpix)*float('NaN')
    tileid = np.ones_like(pix)*float('NaN')
    for i in range(len(data)):
        mask = ind[idx == i]
        #print(i, len(mask), len(np.unique(mask)))
        hptileid[np.unique(mask)] = data['CENTERID'][i] #hp tile center id
        mask2 = np.in1d(pix,mask)
        tileid[np.where(mask2)] = data['CENTERID'][i] #cat tile center id
    
    return keep, tileid

# my colormaps
cm = raichoorlib.mycmap(matplotlib.cm.jet, 10,0,1)
cmr= raichoorlib.mycmap(matplotlib.cm.jet_r,10,0,1)


# mollweide plot setting
# http://balbuceosastropy.blogspot.com/2013/09/the-mollweide-projection.html
def set_mwd(ax,org=0):
    # org is the origin of the plot, 0 or a multiple of 30 degrees in [0,360).
    tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
    tick_labels = np.remainder(tick_labels+360+org,360)
    ax.set_xticklabels(tick_labels)     # we add the scale on the x axis
    ax.set_xlabel('R.A [deg]')
    ax.xaxis.label.set_fontsize(12)
    ax.set_ylabel('Dec. [deg]')
    ax.yaxis.label.set_fontsize(12)
    ax.grid(True)
    return True


# convert radec for mollwide
def get_radec_mw(ra,dec,org):
    ra          = np.remainder(ra+360-org,360) # shift ra values
    ra[ra>180] -= 360    # scale conversion to [-180, 180]
    ra          =- ra    # reverse the scale: East to the left
    return np.radians(ra),np.radians(dec)

# plot/xlim settings
def get_systplot(systquant):
    tmparray = np.array([
        'stardens',      [2.4,3.7],  r'log10(Stellar density from Gaia/dr2 [deg$^{-2}$])',
        'ebv',           [0.01,0.11],'Galactic extinction ebv [mag]',
        'psfsize_g',     [1,2.6],  'g-band psfsize [arcsec]',
        'psfsize_r',     [1,2.6],  'r-band psfsize [arcsec]',
        'psfsize_z',     [1,2.6],  'z-band psfsize [arcsec]',
        'galdepth_g',    [23.3,25.5],'g-band 5sig. galdepth [mag]',
        'galdepth_r',    [23.1,25],'r-band 5sig. galdepth [mag]',
        'galdepth_z',    [21.6,23.9],'z-band 5sig. galdepth [mag]',
        'nobs_g',    [3,4],'g-band NOBS',
        'nobs_r',    [3,4],'r-band NOBS',
        'nobs_z',    [3,4],'z-band NOBS',],
        
        
        dtype='object')
    tmparray = tmparray.reshape(int(tmparray.shape[0]/3),3)
    tmpind   = np.where(tmparray[:,0]==systquant.lower())[0][0]
    return tmparray[tmpind,1], tmparray[tmpind,2]

In [4]:
start = raichoorlib.get_date()
# load catalogue
cat =  get_sweep_whole(dr='dr8-south', rlimit=20, maskbitsource=False, bgsbits=True, opt='1') # catalogue
catindesi = get_isdesi(cat['RA'],cat['DEC']) # True if is in desi footprint
hppix_cat = hp.ang2pix(nside,(90.-cat['DEC'])*np.pi/180.,cat['RA']*np.pi/180.,nest=nest) # catalogue hp pixels array

c = SkyCoord(cat['RA']*units.degree,cat['DEC']*units.degree, frame='icrs')
galb = c.galactic.b.value # galb coordinate
end = raichoorlib.get_date()
print(start)
print(end)

sweep file already exist at:/global/cscratch1/sd/qmxp55/sweep_files/dr8-south_sweep_whole_rlimit_20.npy
Total run time: 1.234354 sec
1207 nearby objects
2019-11-12 13:14:26
2019-11-12 13:15:27


In [5]:
randoms = get_random(N=Nranfiles, sweepsize=None, dr='dr8') # randoms

indesiranfile = '/global/cscratch1/sd/qmxp55/random_dr8_indesi_N%s.npy' %(Nranfiles)
is_indesiranfile = os.path.isfile(indesiranfile)
if is_indesiranfile: 
    ranindesi = np.load(indesiranfile)
else: 
    ranindesi = get_isdesi(randoms['RA'],randoms['DEC']) # True is is in desi footprint
    np.save(indesiranfile, ranindesi)
    
hppixranfile = '/global/cscratch1/sd/qmxp55/hppix_dr8_random_N%s.npy' %(Nranfiles)
is_hppixranfile = os.path.isfile(hppixranfile)
if is_hppixranfile: 
    hppix_ran = np.load(hppixranfile)
else: 
    hppix_ran = hp.ang2pix(nside,(90.-np.array(randoms['DEC']))*np.pi/180.,np.array(randoms['RA'])*np.pi/180.,nest=nest) # randoms hp pixels array
    np.save(hppixranfile, hppix_ran)

RANDOM file already exist at:/global/cscratch1/sd/qmxp55/dr8_random_N3.npy
Total run time: 0.001493 sec
Weight of /global/cscratch1/sd/qmxp55/dr8_random_N3.npy catalogue: 6.82 GB


In [6]:
bgsmask = bgsmask()

In [7]:
rancuts = getGeoCuts(randoms)

In [8]:
def get_dict(cat=None, randoms=None, pixmapfile=None, hppix_ran=None, hppix_cat=None, maskrand=None, maskcat=None, 
                 getnobs=False, nside=None, npix=None, nest=None, pixarea=None, Nranfiles=None, 
                     ranindesi=None, catindesi=None, dec_resol_ns=32.375, namesels=None, galb=None, survey='main'):

    start = raichoorlib.get_date()
    # creating dictionary
    hpdict = {}
    
    if (nside is None) or (npix is None) or (nest is None) or (pixarea is None) & (pixmapfile is not None):
        
        hdr          = fits.getheader(pixmapfile,1)
        nside,nest   = hdr['hpxnside'],hdr['hpxnest']
        npix         = hp.nside2npix(nside)
        pixarea      = hp.nside2pixarea(nside,degrees=True)
    else: raise ValueErro('if not pixel information given, include pixmapfile to compute them.')
    
    if (cat is None) or (pixmapfile is None) or (namesels is None) or (Nranfiles is None):
        raise ValueError('cat, pixmapfile, namesels and Nranfiles can not be None.')
    
    if (getnobs) and (randoms is None):
        raise ValueError('random catalogue can not be None when getnobs is set True.')
        
    if (hppix_ran is None) and (randoms is not None):
        hppix_ran = hp.ang2pix(nside,(90.-np.array(randoms['DEC']))*np.pi/180.,np.array(randoms['RA'])*np.pi/180.,nest=nest)
    elif (hppix_ran is None) and (randoms is None):
        raise ValueError('include a random catalogue to compute their hp pixels indexes.')
        
    if (hppix_cat is None):
        hppix_cat = hp.ang2pix(nside,(90.-cat['DEC'])*np.pi/180.,cat['RA']*np.pi/180.,nest=nest) # catalogue hp pixels array
    
    if (ranindesi is None) and (randoms is not None):
        ranindesi = get_isdesi(randoms['RA'],randoms['DEC']) # True is is in desi footprint
    elif (ranindesi is None) and (randoms is None):
        raise ValueError('include a random catalogue to compute ranindesi.')
        
    if (catindesi is None):
        catindesi = get_isdesi(cat['RA'],cat['DEC']) # True is is in desi footprint
        
    if galb is None:
        c = SkyCoord(cat['RA']*units.degree,cat['DEC']*units.degree, frame='icrs')
        galb = c.galactic.b.value # galb coordinate
        
    theta,phi  = hp.pix2ang(nside,np.arange(npix),nest=nest)
    hpdict['ra'],hpdict['dec'] = 180./np.pi*phi,90.-180./np.pi*theta
    c = SkyCoord(hpdict['ra']*units.degree,hpdict['dec']*units.degree, frame='icrs')
    hpdict['gall'],hpdict['galb'] = c.galactic.l.value,c.galactic.b.value

    # is in desi tile?
    hpdict['isdesi'] = get_isdesi(hpdict['ra'],hpdict['dec'])
    print('positions and desifotprint DONE...')

    # propagating some keys from ADM pixweight
    hdu = fits.open(pixmapfile)
    data = hdu[1].data
    for key in ['HPXPIXEL', 'FRACAREA', 
            'STARDENS', 'EBV', 
            'PSFDEPTH_G', 'PSFDEPTH_R', 'PSFDEPTH_Z',
            'GALDEPTH_G', 'GALDEPTH_R', 'GALDEPTH_Z',
            'PSFDEPTH_W1', 'PSFDEPTH_W2',
            'PSFSIZE_G', 'PSFSIZE_R', 'PSFSIZE_Z']:
        if (key=='STARDENS'):
            hpdict[key.lower()] = np.log10(data[key])
        elif (key[:8]=='GALDEPTH'):
            hpdict[key.lower()] = 22.5-2.5*np.log10(5./np.sqrt(data[key]))
        else:
            hpdict[key.lower()] = data[key]
    print('systematics DONE...')
        
    # computing fracareas
    randdens = 5000*Nranfiles
    print('randdens = ', randdens, ' ; len randoms = ', len(hppix_ran))
    if maskrand is None:
        ind,c           = np.unique(hppix_ran[ranindesi],return_counts=True)
    else:
        ind,c           = np.unique(hppix_ran[(maskrand) & (ranindesi)],return_counts=True)
    hpdict['bgsfracarea']      = np.zeros(npix)
    hpdict['bgsfracarea'][ind] = c / randdens / pixarea
    print('bgsfracarea DONE...')
    
    # computing nobs
    if getnobs:
        import pandas as pd
        if maskrand is None:
            s = pd.Series(hppix_ran[ranindesi])
        else:
            s = pd.Series(hppix_ran[(maskrand) & (ranindesi)])
        d = s.groupby(s).groups
        for i in ['NOBS_G', 'NOBS_R', 'NOBS_Z']:
            hpdict[i] = np.zeros(npix)
            for j in d.keys():
                hpdict[i][j] = np.mean(randoms[i][d[j]])
        print('nobs DONE...')
        
    # north/south/des/decals
    hpdict['issouth'] = np.zeros(npix,dtype=bool)
    tmp               = (hpdict['bgsfracarea']>0) & ((hpdict['galb']<0) | ((hpdict['galb']>0) & (hpdict['dec']<dec_resol_ns)))
    hpdict['issouth'][tmp] = True
    hpdict['isnorth'] = np.zeros(npix,dtype=bool)
    tmp               = (hpdict['bgsfracarea']>0) & (hpdict['dec']>dec_resol_ns) & (hpdict['galb']>0)
    hpdict['isnorth'][tmp] = True
    hpdict['isdes']   = raichoorlib.get_isdes(hpdict['ra'],hpdict['dec'])
    hpdict['isdecals'] = (hpdict['issouth']) & (~hpdict['isdes'])
    hpdict['istest'] = (hpdict['ra'] > 160.) & (hpdict['ra'] < 230.) & (hpdict['dec'] > -2.) & (hpdict['dec'] < 18.)
    print('regions DONE...')

    # areas
    hpdict['area_all']   = hpdict['bgsfracarea'].sum() * pixarea
    for reg in ['south','decals','des','north', 'test']:
        hpdict['bgsarea_'+reg]   = hpdict['bgsfracarea'][hpdict['is'+reg]].sum() * pixarea
    print('areas DONE...')
    
    #target densities
    #namesels = {'any':-1, 'bright':1, 'faint':0, 'wise':2}
    for foot in ['north','south']:
        
        data = cat
        
        if (foot=='north'): keep = (data['DEC']>dec_resol_ns) & (galb>0)
        if (foot=='south'): keep = (data['DEC']<dec_resol_ns) | (galb<0)        
        ##
        if maskcat is None: keep &= catindesi
        else: keep &= (maskcat) & (catindesi)
        
        if survey == 'main': bgstargetname = 'BGS_TARGET'
        elif survey == 'sv1': bgstargetname = 'SV1_BGS_TARGET'
        elif survey == 'bgs': bgstargetname = 'BGSBITS'

        for namesel, bitnum in zip(namesels.keys(), namesels.values()):
            print('computing for ', foot, '/', namesel)
            if (namesel=='any'):             sel = np.ones(len(data),dtype=bool)
            else:                            sel = ((data[bgstargetname] & 2**(bitnum)) != 0)
            
            ind,c = np.unique(hppix_cat[(sel) & (keep)],return_counts=True)
            hpdict[foot+'_n'+namesel]      = np.zeros(npix)
            hpdict[foot+'_n'+namesel][ind] = c
        print('target densities in %s DONE...' %(foot))
            
    # storing mean hpdens
    isdesi = (hpdict['isdesi']) & (hpdict['bgsfracarea']>0)
    for namesel in namesels.keys():
        ## south + north density
        hpdens = (hpdict['south_n'+namesel] + hpdict['north_n'+namesel] ) / (pixarea * hpdict['bgsfracarea'])
        ## split per region
        for reg in ['all','des','decals','north', 'south', 'test']:
            if (reg=='all'):
                hpdict['meandens_'+namesel+'_'+reg] = np.nanmean(hpdens[isdesi])
            else:
                hpdict['meandens_'+namesel+'_'+reg] = np.nanmean(hpdens[(isdesi) & (hpdict['is'+reg])])
            print('meandens_'+namesel+'_'+reg+' = '+'%.0f'%hpdict['meandens_'+namesel+'_'+reg]+' /deg2')
            
    # storing total target density
    isdesi = (hpdict['isdesi']) & (hpdict['bgsfracarea']>0)
    for namesel in namesels.keys():
        ## split per region
        for reg in ['all','des','decals','north', 'south', 'test']:
            if (reg=='all'):
                hpdict['dens_'+namesel+'_'+reg] = (hpdict['south_n'+namesel][isdesi] + hpdict['north_n'+namesel][isdesi]).sum() / (pixarea * hpdict['bgsfracarea'][isdesi].sum())
            else:
                hpdict['dens_'+namesel+'_'+reg] = (hpdict['south_n'+namesel][(isdesi) & (hpdict['is'+reg])] + hpdict['north_n'+namesel][(isdesi) & (hpdict['is'+reg])]).sum() / (pixarea * hpdict['bgsfracarea'][(isdesi) & (hpdict['is'+reg])].sum())
            print('dens_'+namesel+'_'+reg+' = '+'%.0f'%hpdict['dens_'+namesel+'_'+reg]+' /deg2')
        
    end = raichoorlib.get_date()
    print('start:', start)
    print('end:', end)
    
    return hpdict
    

In [9]:
hpdict = get_dict(cat=cat, pixmapfile=inptfiles['dr8pix'], hppix_ran=hppix_ran, hppix_cat=hppix_cat, 
                      maskrand=None, maskcat=None, Nranfiles=Nranfiles, ranindesi=ranindesi, catindesi=catindesi, 
                            namesels=bgsmask, galb=galb, survey='bgs')

1207 nearby objects
positions and desifotprint DONE...
systematics DONE...
randdens =  15000  ; len randoms =  304987125
bgsfracarea DONE...
regions DONE...
areas DONE...
computing for  north / BS
computing for  north / MS
computing for  north / GC
computing for  north / LG
computing for  north / allmask
computing for  north / nobs
computing for  north / SG
computing for  north / FMC
computing for  north / CC
computing for  north / QC_FM
computing for  north / QC_FI
computing for  north / QC_FF
computing for  north / QC_IVAR
target densities in north DONE...
computing for  south / BS
computing for  south / MS
computing for  south / GC
computing for  south / LG
computing for  south / allmask
computing for  south / nobs
computing for  south / SG
computing for  south / FMC
computing for  south / CC
computing for  south / QC_FM
computing for  south / QC_FI
computing for  south / QC_FF
computing for  south / QC_IVAR
target densities in south DONE...
meandens_BS_all = 3394 /deg2
meandens_BS_

In [10]:
ranin = np.load('/global/cscratch1/sd/qmxp55/inREGIONS_randoms_dr8_0.31.1_N3.npy')
ranin.dtype.descr

[('isdes', '|b1'),
 ('isdecals', '|b1'),
 ('isnorth', '|b1'),
 ('isdesi', '|b1'),
 ('istest', '|b1')]

In [11]:
#
def get_reg(reg='decals', hpdict=hpdict, hppix_cat=hppix_cat, hppix_ran=hppix_ran):
    
    isreg_pixlist = hpdict['hpxpixel'][hpdict['is'+reg]]
    regcat = np.in1d(hppix_cat, isreg_pixlist)
    #regran = np.in1d(hppix_ran, isreg_pixlist)
    
    return regcat

catindecals = get_reg(reg='decals', hpdict=hpdict, hppix_cat=hppix_cat, hppix_ran=hppix_ran)
ranindecals = ranin['isdecals']

In [14]:
def getStats(cat=cat, hpdict=hpdict, bgsmask=bgsmask, rancuts=rancuts, CurrentMask=['BS', 'MS', 'LG', 'GC'], PrevMask=['SG'], 
                 reg='decals', regcat=catindecals, regran=ranindecals):
    
    from astropy.table import Table
    Tab = []
    GMT = np.zeros_like(catindesi, dtype='?')
    GMT_ran = np.zeros_like(ranindesi, dtype='?')
    PMT = GMT.copy()
    PMT_ran = GMT_ran.copy()
        
    #if (regcat is not None) & (regran is not None): print('region set...')
    #elif (regcat is None) & (regran is None): regcat, regran = ~GMT.copy(), ~GMT_ran.copy()
    #else: raise ValueError('regcat and regran both have to be None-type or non None-type.')
     
    # area in region
    Areg = hpdict['bgsarea_'+reg]
    # Number of randoms in region
    NRreg = np.sum(regran)
    #
    B, F = cat['RMAG'] < 19.5, np.logical_and(cat['RMAG'] < 20, cat['RMAG'] > 19.5)
        
    if PrevMask is not None:
        PM_lab = '|'.join(PrevMask)
        for i in PrevMask:
            PMT |= (cat['BGSBITS'] & 2**(bgsmask[i])) == 0
            if i in rancuts.keys(): PMT_ran |= ~rancuts[i]   
    else:
        PM_lab = 'None'
    
    for i in CurrentMask:
        
        if i in rancuts.keys(): A_i = (np.sum((~rancuts[i] & (~PMT_ran) & (regran)))/NRreg)*(Areg)
        else: A_i = 0.
        bgscut = (cat['BGSBITS'] & 2**(bgsmask[i])) == 0
        #eta_B_i_in = np.sum((GeoCutsDict[i]) & (B) & (~PMT))/(A_i) #density over the geometric area
        #eta_F_i_in = np.sum((GeoCutsDict[i]) & (F) & (~PMT))/(A_i) #density over the geometric area
        eta_B_i = np.sum((bgscut) & (B) & (~PMT) & (regcat))/(Areg) #density over the total area
        eta_F_i = np.sum((bgscut) & (F) & (~PMT) & (regcat))/(Areg) #density over the total area
        
        Tab.append([i, round(A_i*(100/Areg), 2), round(eta_B_i,2), round(eta_F_i,2)])
            
        GMT |= bgscut
        if i in rancuts.keys(): GMT_ran |= ~rancuts[i]  
    
    lab = '|'.join(CurrentMask)
    lab_in = '(%s)' %(lab)
    lab_out = '~(%s)*' %(lab)
    lab_out2 = '~(%s)' %(lab)
    
    A_GMT_in = (np.sum((GMT_ran) & (~PMT_ran) & (regran))/NRreg)*(Areg)
    eta_B_GMT_in_1 = np.sum((GMT) & (B) & (~PMT) & (regcat))/(Areg) #Not corrected for mask area
    eta_F_GMT_in_1 = np.sum((GMT) & (F) & (~PMT) & (regcat))/(Areg) #Not corrected for mask area

    A_GMT_out = (np.sum((~GMT_ran) & (~PMT_ran) & (regran))/NRreg)*(Areg)
    eta_B_GMT_out_1 = np.sum((~GMT) & (B) & (~PMT) & (regcat))/(Areg) #Not corrected for mask area
    eta_B_GMT_out_2 = np.sum((~GMT) & (B) & (~PMT) & (regcat))/(A_GMT_out) #Corrected for mask area
    eta_F_GMT_out_1 = np.sum((~GMT) & (F) & (~PMT) & (regcat))/(Areg) #Not corrected for mask area
    eta_F_GMT_out_2 = np.sum((~GMT) & (F) & (~PMT) & (regcat))/(A_GMT_out) #Corrected for mask area
    
    if len(CurrentMask) > 1:
        Tab.append([lab_in, round(A_GMT_in*(100/Areg),2), round(eta_B_GMT_in_1,2), round(eta_F_GMT_in_1,2)])
    Tab.append([lab_out, round(A_GMT_out*(100/Areg),2), round(eta_B_GMT_out_1,2), round(eta_F_GMT_out_1,2)])
    Tab.append([lab_out2, round(A_GMT_out*(100/Areg),2), round(eta_B_GMT_out_2,2), round(eta_F_GMT_out_2,2)])
    
    Tab = np.transpose(Tab)
    t = Table([Tab[0], Tab[1], Tab[2], Tab[3]], 
              names=('GM','$f_{A}$ [$\%$]', '$\eta_{B}$ [deg$^2$]', '$\eta_{F}$ [deg$^2$]'),
                    dtype=('S', 'f8', 'f8', 'f8'))
    
    print('Previous Cuts: (%s)' %(PM_lab))
    print('Current Cuts: %s' %(lab_in))
                                    
    return t


In [15]:
t = getStats(cat=cat, hpdict=hpdict, bgsmask=bgsmask, rancuts=rancuts, CurrentMask=['BS', 'LG', 'FMC'], PrevMask=['SG'], 
                 reg='decals', regcat=catindecals, regran=ranindecals)

Previous Cuts: (SG)
Current Cuts: (BS|LG|FMC)


In [72]:
t

GM,$f_{A}$ [$\%$],$\eta_{B}$ [deg$^2$],$\eta_{F}$ [deg$^2$]
bytes13,float64,float64,float64
BS,2.83,14.75,8.83
LG,0.08,21.84,1.45
FMC,0.0,27.45,13.4
(BS|LG|FMC),2.9,60.04,23.34
~(BS|LG|FMC)*,97.1,801.04,567.8
~(BS|LG|FMC),97.1,825.0,584.78


In [70]:
#
from photometric_def import masking, results
import pygraphviz as pgv
from PIL import Image

def flow(cat=cat, hpdict=hpdict, bgsmask=bgsmask, rancuts=rancuts, order=None, reg='decals', 
             regcat=catindecals, regran=ranindecals, file=None):
    
    if order is None:
        raise ValueError('define the order of flow chart.')
    
    T = Table()
    Areg = hpdict['bgsarea_'+reg]
    
    B, F = cat['RMAG'] < 19.5, np.logical_and(cat['RMAG'] < 20, cat['RMAG'] > 19.5)
    den0B = np.sum((regcat) & (B))/Areg
    den0F = np.sum((regcat) & (F))/Areg
    
    #T['SU'] = masking(title='START', submasks=None, details=None)
    #T['SG'] = masking(title='GEOMETRICAL', submasks=None, details=None)
    T['I'] = masking(title='%s (%s)' %('LS DR8',reg.upper()), submasks=['rmag < %2.2g' %(20)], details=None)
    T['RI'] = results(a=Areg, b=den0B, f=den0F, stage='ini', per=False)
    
    G=pgv.AGraph(strict=False,directed=True)

    elist = []
    rejLab = []
    #define initial params in flow chart
    #ini = ['SU', 'I', 'RI', 'SG']
    ini = ['I', 'RI']
    for i in range(len(ini) - 1):
        elist.append((list(T[ini[i]]),list(T[ini[i+1]])))
        
    #G.add_edges_from(elist)
    #stages=['SU', 'SG']
    #G.add_nodes_from([list(T[i]) for i in stages], color='green', style='filled')
    nlist=['RI']
    G.add_nodes_from([list(T[i]) for i in nlist], color='lightskyblue', shape='box', style='filled')
    maskings=['I']
    G.add_nodes_from([list(T[i]) for i in maskings], color='lawngreen', style='filled')
        
    #
    for num, sel in enumerate(order):
        
        T['I'+str(num)] = masking(title=' & '.join(sel), submasks=None, details=None)
        
        if num == 0: elist.append((list(T['I']),list(T['I'+str(num)])))
        else: elist.append((list(T['R'+str(num-1)]),list(T['I'+str(num)])))
            
        if num == 0: pm = None
        elif num == 1: pm = order[0]
        else: pm += order[num-1]
            
        if len(sel) > 1: IGMLab_2 = ' | '.join(sel)
        else: IGMLab_2 = sel[0]
        
        t = getStats(cat=cat, hpdict=hpdict, bgsmask=bgsmask, rancuts=rancuts, CurrentMask=sel, PrevMask=pm, 
                 reg=reg, regcat=regcat, regran=regran)
        
        T['R'+str(num)] = results(a=t[-2][1], b=t[-2][2], f=t[-2][3], b2=t[-1][2], f2=t[-1][3], stage='geo', per=True)
        T['REJ'+str(num)] = results(a=t[-3][1], b=t[-3][2], f=t[-3][3], stage='ini', per=True, title='(%s)' %(IGMLab_2))
        
        elist.append((list(T['I'+str(num)]),list(T['REJ'+str(num)])))
        elist.append((list(T['I'+str(num)]),list(T['R'+str(num)])))
        
        if False in [i in rancuts.keys() for i in sel]: icolor = 'plum'
        else: icolor = 'lightgray'
        
        Rlist=['R'+str(num)]
        G.add_nodes_from([list(T[i]) for i in Rlist], color='lightskyblue', shape='box', style='filled')
        REJlist=['REJ'+str(num)]
        G.add_nodes_from([list(T[i]) for i in REJlist], color='lightcoral', shape='box', style='filled')
        Ilist=['I'+str(num)]
        G.add_nodes_from([list(T[i]) for i in Ilist], color=icolor, style='filled')

        if len(sel) > 1:
            for i, j in enumerate(sel):
                T['REJ'+str(num)+str(i)] = results(a=t[i][1], b=t[i][2], f=t[i][3], stage='ini', per=True, title=j)
                elist.append((list(T['REJ'+str(num)]),list(T['REJ'+str(num)+str(i)])))
                
                REJilist=['REJ'+str(num)+str(i)]
                G.add_nodes_from([list(T[i]) for i in REJilist], color='coral', shape='box', style='filled')
        
    #
    if file is None:
        pathdir = os.getcwd()+'/'+'results'+'_'+reg
        if not os.path.isdir(pathdir): os.makedirs(pathdir)
        file = pathdir+'/'+'flow'
        
    G.add_edges_from(elist)
    G.write('%s.dot' %(file)) # write to simple.dot
    BB=pgv.AGraph('%s.dot' %(file)) # create a new graph from file
    BB.layout(prog='dot') # layout with default (neato)
    BB.draw('%s.png' %(file)) # draw png
    #os.system('convert ' + file + '.ps ' + file + '.png')
    flow = Image.open('%s.png' %(file))

    return flow, elist, T
    

In [74]:
#
order = [['SG'], ['BS', 'LG', 'GC', 'MS'], ['nobs'], ['QC_FM', 'QC_FI', 'QC_FF', 'QC_IVAR'], ['FMC', 'CC']]
#order = [['SG'], ['BS', 'MS'], ['FMC', 'CC']]
flow, _, _ = flow(cat=cat, hpdict=hpdict, bgsmask=bgsmask, rancuts=rancuts, order=order, reg='decals', 
             regcat=catindecals, regran=ranindecals, file=None)

flow

TypeError: 'PngImageFile' object is not callable

In [75]:
#
order = [['BS', 'LG', 'GC', 'MS'], ['nobs'], ['SG'], ['FMC', 'CC'], ['QC_FM', 'QC_FI', 'QC_FF', 'QC_IVAR']]
#order = [['SG'], ['BS', 'MS'], ['FMC', 'CC']]
flow2, _, _ = flow(cat=cat, hpdict=hpdict, bgsmask=bgsmask, rancuts=rancuts, order=order, reg='decals', 
             regcat=catindecals, regran=ranindecals, file=None)

flow2

TypeError: 'PngImageFile' object is not callable